# Files duplicate analysis

There are several cases when file duplicates should be identified.
For instance, I need to know if I already have e-book in my library.
Let's use Pandas to find inconsistency. 

In [2]:
from pathlib import Path
import pandas as pd

In [53]:
main_path = r"C:\Users\Admin\Downloads"
compare_path = 'D:\Books'
duplicates_path = r"C:\Users\Admin\Downloads\_duplicates"

In [47]:
def get_files_in_folder(folder: Path | str):
    if type(folder) == str:
        folder = Path(folder)
    return [f for f in folder.iterdir() if f.is_file()]

def get_files_tree(path: Path | str):
    if type(path) == str:
        path = Path(path)
    
    files_list = get_files_in_folder(path)
    for folder in path.iterdir():
        if folder.is_dir():
            # print(path, '\\', folder.name)
            # print(get_files_in_folder(folder))
            files_list.extend(get_files_in_folder(folder))
            get_files_tree(folder)
    return files_list


In [48]:
import re
def clear_name(name: str):
    value = re.sub("[!#]", "", name)
    return re.sub('[+_-]', ' ', value).lstrip()

# clear_name('+_test_data_#.pdf')

In [49]:
titles = ['File', 'Original', 'Path']
df_main = pd.DataFrame(columns = titles)

# files = get_files_in_folder(main_path)
files = get_files_tree(main_path)
for file in files:
    df_main.loc[len(df_main)] = [clear_name(file.name), file.name, file.absolute()]

df_compare = pd.DataFrame(columns = titles)
files = get_files_tree(compare_path)
for file in files:
    df_compare.loc[len(df_compare)] = [clear_name(file.name), file.name, file.absolute()]


In [52]:
df = df_main.merge(df_compare, how='inner', on='File', suffixes=('_main', '_compare'))
df_merged = df[df["Path_main"] != df["Path_compare"]]
# df_merged

In [51]:
if not Path(duplicates_path).exists():
    Path(duplicates_path).mkdir()

moved = 0
for item in range(0, len(df_merged)):
    location: Path = df_merged.loc[item]['Path_main']
    destination = Path(duplicates_path + '\\' + location.name)
    location.rename(destination)
    moved += 1

print(f'Into folder "{duplicates_path}" moved {moved} file(s)') 
  

Into folder "C:\Users\Admin\Downloads\_duplicates" moved 1 file(s)
